## Zillow API to make a database

In this tutorial we are going to use an API to get data from zillow to create a database. For the first step you will have to sign up with Zillow for an API, they will give you a ZWSID key that you will need to keeep. Once you do that you are going use the equests library to type in the address of the API to make a call. Once you do that you will get the following result:

In [1]:
import requests 
import urllib.parse
_ZWSID=open(r"C:\Users\sespejog\OneDrive\Data Sets\LA COUNTY DATA\Real Estate Data Mining").readline()
#_address=urllib.parse.quote_plus("5190 Ithaca Ave")
#city_zip=urllib.parse.quote_plus("Los Angeles CA 90032")
_address=urllib.parse.quote_plus("51 Exeter") #we url encode our address because that is how zillow wants us to input our data
city_zip=urllib.parse.quote_plus("Irvine CA 92612")
_params={"zws-id":_ZWSID,"address":_address,"citystatezip":city_zip}#we put our inpput parameters into a dictionary 
r= requests.get("http://www.zillow.com/webservice/GetDeepSearchResults.htm",params=_params)#Here is where python makes the request
r.content

b'<?xml version="1.0" encoding="utf-8"?><SearchResults:searchresults xsi:schemaLocation="http://www.zillow.com/static/xsd/SearchResults.xsd http://www.zillowstatic.com/vstatic/80d5e73/static/xsd/SearchResults.xsd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:SearchResults="http://www.zillow.com/static/xsd/SearchResults.xsd"><request><address>51+Exeter</address><citystatezip>Irvine+CA+92612</citystatezip></request><message><text>Request successfully processed</text><code>0</code></message><response><results><result><zpid>25759925</zpid><links><homedetails>http://www.zillow.com/homedetails/51-Exeter-Irvine-CA-92612/25759925_zpid/</homedetails><graphsanddata>http://www.zillow.com/homedetails/51-Exeter-Irvine-CA-92612/25759925_zpid/#charts-and-data</graphsanddata><mapthishome>http://www.zillow.com/homes/25759925_zpid/</mapthishome><comparables>http://www.zillow.com/homes/comps/25759925_zpid/</comparables></links><address><street>51 Exeter</street><zipcode>92612</zipcode><cit

Our response from the API is in xml format, a data structure that looks similar to html. If we want to parse it we will need to import the built in anaconda xml library. We then will create a root branch using our xml file.

In [2]:
import xml.etree.ElementTree as ET
root=ET.fromstring(r.content)
for child in root:
    print(child.tag, child.attrib)#we are printing out the attribute


request {}
message {}
response {}


The data is nested inside the last branch, so we will have to write a code to go inside each node and show us its output.

In [4]:

for node in root[2].iter():
    print(node.tag,"*"*10,node.text)

response ********** None
results ********** None
result ********** None
zpid ********** 25759925
links ********** None
homedetails ********** http://www.zillow.com/homedetails/51-Exeter-Irvine-CA-92612/25759925_zpid/
graphsanddata ********** http://www.zillow.com/homedetails/51-Exeter-Irvine-CA-92612/25759925_zpid/#charts-and-data
mapthishome ********** http://www.zillow.com/homes/25759925_zpid/
comparables ********** http://www.zillow.com/homes/comps/25759925_zpid/
address ********** None
street ********** 51 Exeter
zipcode ********** 92612
city ********** Irvine
state ********** CA
latitude ********** 33.65125
longitude ********** -117.827541
FIPScounty ********** 6059
useCode ********** Condominium
taxAssessmentYear ********** 2018
taxAssessment ********** 565656.0
yearBuilt ********** 1981
finishedSqFt ********** 1242
bathrooms ********** 3.0
bedrooms ********** 2
lastSoldDate ********** 09/17/2007
lastSoldPrice ********** 490000
zestimate ********** None
amount ********** 620429
l

### How would a bad response look like?

It is important to know how a bad request looks back so when we merge our data we don't get any unexpected errors. Let's try inputing a ficticious error in our API and see what it gives us.

In [13]:
_address=urllib.parse.quote_plus("51 Exeturd") #we url encode our address because that is how zillow wants us to input our data
city_zip=urllib.parse.quote_plus("Irvine CA 92612")
_params={"zws-id":_ZWSID,"address":_address,"citystatezip":city_zip}#we put our inpput parameters into a dictionary 
r= requests.get("http://www.zillow.com/webservice/GetDeepSearchResults.htm",params=_params)#Here is where python makes the request
root=ET.fromstring(r.content)
for node in root[1].iter():
    print(repr(node.tag),node.text)


'message' None
'text' Error: no exact match found for input address
'code' 508


Now that we now how our data looks like we can create a data structure that python can make into a table for us.
    

In [14]:
import pandas as pd
pd.set_option('display.max_columns', 999) #this makes sure we can see all our columns
information={}
if len(root)==3:
    for node in root[2].iter():
        if type(node.text)!= type(None):
            if "http://" not in node.text:
                information[node.tag]=node.text
    information["full_address"]="{0:} {1:} {2:} {3:}".format(information["street"],information["city"],information["state"],information["zipcode"])
    zillow_data=pd.DataFrame(information,index=[0])
zillow_data

,zpid,street,zipcode,city,state,latitude,longitude,FIPScounty,useCode,taxAssessmentYear,taxAssessment,yearBuilt,finishedSqFt,bathrooms,bedrooms,lastSoldDate,lastSoldPrice,amount,last-updated,valueChange,low,high,percentile,zindexValue,full_address
0,25759925,51 Exeter,92612,Irvine,CA,33.65125,-117.827541,6059,Condominium,2018,565656.0,1981,1242,3.0,2,09/17/2007,490000,620429,10/26/2019,-1638,589408,651450,0,"532,300",51 Exeter Irvine CA 92612


Now that we know how to get information from the Zillow API we are going to use our LA County Database to push some requests.

In [75]:
pd.set_option('display.max_columns', 999)
LA_County=pd.read_csv(r"C:\Users\sespejog\OneDrive\Data Sets\LA COUNTY DATA\real_estate.csv")
LA_County.head(5)

,Unnamed: 0,ZIPcode,TaxRateArea_CITY,AIN,RollYear,TaxRateArea,AssessorID,PropertyLocation,PropertyType,PropertyUseCode,GeneralUseType,SpecificUseType,SpecificUseDetail1,SpecificUseDetail2,totBuildingDataLines,YearBuilt,EffectiveYearBuilt,SQFTmain,Bedrooms,Bathrooms,Units,RecordingDate,LandValue,LandBaseYear,ImprovementValue,ImpBaseYear,TotalLandImpValue,HomeownersExemption,RealEstateExemption,FixtureValue,FixtureExemption,PersonalPropertyValue,PersonalPropertyExemption,isTaxableParcel?,TotalValue,TotalExemption,netTaxableValue,SpecialParcelClassification,AdministrativeRegion,Cluster,ParcelBoundaryDescription,HouseNo,HouseFraction,StreetDirection,StreetName,UnitNo,City,ZIPcode5,rowID,CENTER_LAT,CENTER_LON,Location 1
0,0,90031-1636,LOS ANGELES,5304006004,2019,4,5304-006-004,1004 MONTECITO DR LOS ANGELES CA 90031,SFR,0100,Residential,Single Family Residence,NaN,NaN,1,1962,1962,1389,3,2,1,19980626,117363,1998,74766,1998,192129,7000,0,0,0,0,0,Y,192129,7000,185129,NaN,4,4102,TRACT NO 1O11O LOT 24,1004,NaN,NaN,MONTECITO DR,NaN,LOS ANGELES CA,90031,20195304006004,34.087768,-118.203246,"[None, '34.08776772', '-118.20324594', None, F..."
1,1,90031-1635,LOS ANGELES,5304008001,2019,4,5304-008-001,1007 MONTECITO DR LOS ANGELES CA 90031,SFR,0100,Residential,Single Family Residence,NaN,NaN,1,2002,2002,1844,3,3,1,20120523,198640,2002,218505,2002,417145,7000,0,0,0,0,0,Y,417145,7000,410145,NaN,4,4102,TRACT NO 1O11O LOT 37,1007,NaN,NaN,MONTECITO DR,NaN,LOS ANGELES CA,90031,20195304008001,34.087744,-118.202571,"[None, '34.08774435', '-118.20257092', None, F..."
2,2,90031-1636,LOS ANGELES,5304006005,2019,4,5304-006-005,1008 MONTECITO DR LOS ANGELES CA 90031,SFR,0100,Residential,Single Family Residence,NaN,NaN,1,1962,1962,1389,2,2,1,20070629,480414,2007,144119,2007,624533,7000,0,0,0,0,0,Y,624533,7000,617533,NaN,4,4102,TRACT NO 1O11O LOT 25,1008,NaN,NaN,MONTECITO DR,NaN,LOS ANGELES CA,90031,20195304006005,34.087652,-118.203220,"[None, '34.08765199', '-118.20321957', None, F..."
3,3,90031-2070,LOS ANGELES,5205021027,2019,4,5205-021-027,100 E AVENUE 28 LOS ANGELES CA 90031,C/I,2670,Commercial,"Auto, Recreation Equipment, Construction Equip...",Auto Service Centers (No Gasoline),One Story,1,1941,1956,1108,0,0,0,20131121,39531,1975,39805,1975,79336,0,0,0,0,0,0,Y,79336,0,79336,NaN,27,27629,H T HAZARD'S SUB OF LOT 16 GRIFFINS ADD TO E L...,100,NaN,E,AVENUE 28,NaN,LOS ANGELES CA,90031,20195205021027,34.079034,-118.214673,"[None, '34.07903447', '-118.21467277', None, F..."
4,4,90031-1636,LOS ANGELES,5304006006,2019,4,5304-006-006,1014 MONTECITO DR LOS ANGELES CA 90031,SFR,0100,Residential,Single Family Residence,NaN,NaN,1,1966,1966,1360,3,2,1,20170802,74956,1986,112451,1986,187407,7000,0,0,0,0,0,Y,187407,7000,180407,NaN,4,4102,TRACT NO 1O11O LOT 26,1014,NaN,NaN,MONTECITO DR,NaN,LOS ANGELES CA,90031,20195304006006,34.087541,-118.203234,"[None, '34.08754092', '-118.20323376', None, F..."


In [15]:
## We are going to filter by residential homes
LA_Residential=LA_County[(LA_County["GeneralUseType"]=="Residential")&(LA_County["EffectiveYearBuilt"]>1990)&(LA_County["SpecificUseType"]=="Single Family Residence")]
LA_Residential.shape

(444, 52)

## Using a Dataframe to feed Data to Zillow API

Here is the first result. Once we get it down we will use it for every single house

In [25]:
string=LA_Residential.iloc[5,:]
new_address=str(string["HouseNo"])+" "+ str(string["StreetName"])
new_city_zip=str(string["City"])+" "+ str(string["ZIPcode5"])

_params={"zws-id":_ZWSID,"address":new_address,"citystatezip":new_city_zip}#we put our inpput parameters into a dictionary 
r= requests.get("http://www.zillow.com/webservice/GetDeepSearchResults.htm",params=_params)#Here is where python makes the request
r.content

b'<?xml version="1.0" encoding="utf-8"?><SearchResults:searchresults xsi:schemaLocation="http://www.zillow.com/static/xsd/SearchResults.xsd http://www.zillowstatic.com/vstatic/80d5e73/static/xsd/SearchResults.xsd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:SearchResults="http://www.zillow.com/static/xsd/SearchResults.xsd"><request><address>1226 MONTECITO DR</address><citystatezip>LOS ANGELES CA 90031</citystatezip></request><message><text>Request successfully processed</text><code>0</code></message><response><results><result><zpid>20689168</zpid><links><homedetails>http://www.zillow.com/homedetails/1226-Montecito-Dr-Los-Angeles-CA-90031/20689168_zpid/</homedetails><graphsanddata>http://www.zillow.com/homedetails/1226-Montecito-Dr-Los-Angeles-CA-90031/20689168_zpid/#charts-and-data</graphsanddata><mapthishome>http://www.zillow.com/homes/20689168_zpid/</mapthishome><comparables>http://www.zillow.com/homes/comps/20689168_zpid/</comparables></links><address><street>1226 Mo

In [63]:

def get_info(number,street,city_state,zip_code):
    '''feed data to zillow returns xml string'''
    new_address=str(number)+" "+ str(street)
    new_city_zip=str(city_state)+" "+ str(zip_code)
    _params={"zws-id":_ZWSID,"address":new_address,"citystatezip":new_city_zip}#we put our inpput parameters into a dictionary 
    r= requests.get("http://www.zillow.com/webservice/GetDeepSearchResults.htm",params=_params)#Here is where python makes the request
    return r.content

def parse_xml(xml_string,number,street,city_state,zip_code):
    root=ET.fromstring(xml_string)
    information={}
    outer_dict={}
    if len(root)==3:
        for node in root[2].iter():
            if type(node.text)!= type(None):
                if "http://" not in node.text:
                    information[node.tag]=node.text
        outer_dict["{0:} {1:} {2:} {3:}".format(number,street,city_state,zip_code)]=information
    return outer_dict

xml_test=get_info(string["HouseNo"],string["StreetName"],string["City"],string["ZIPcode5"])
parse_xml(xml_test,string["HouseNo"],string["StreetName"],string["City"],string["ZIPcode5"])

{'1226 MONTECITO DR LOS ANGELES CA 90031': {'zpid': '20689168',
  'street': '1226 Montecito Dr',
  'zipcode': '90031',
  'city': 'Los Angeles',
  'state': 'CA',
  'latitude': '34.087346',
  'longitude': '-118.198436',
  'FIPScounty': '6037',
  'useCode': 'SingleFamily',
  'taxAssessmentYear': '2019',
  'taxAssessment': '540908.0',
  'yearBuilt': '1993',
  'lotSizeSqFt': '5924',
  'finishedSqFt': '1471',
  'bathrooms': '3.0',
  'bedrooms': '3',
  'lastSoldDate': '05/31/2013',
  'lastSoldPrice': '490000',
  'amount': '764688',
  'last-updated': '10/26/2019',
  'valueChange': '-7486',
  'low': '703513',
  'high': '833510',
  'percentile': '0',
  'zindexValue': '535,500'}}

## Now we use our functions on our LA Residential Dataframe



In [90]:
Main_List=[]
for i in LA_Residential.index:
    xml_test=get_info(LA_Residential.loc[i,"HouseNo"],LA_Residential.loc[i,"StreetName"],LA_Residential.loc[i,"City"],LA_Residential.loc[i,"ZIPcode5"])
    Main_List.append(parse_xml(xml_test,LA_Residential.loc[i,"HouseNo"],LA_Residential.loc[i,"StreetName"],LA_Residential.loc[i,"City"],LA_Residential.loc[i,"ZIPcode5"]))


In [92]:
Main_List[:5]

[{'1007 MONTECITO DR LOS ANGELES CA 90031': {'zpid': '20689097',
   'street': '1007 Montecito Dr',
   'zipcode': '90031',
   'city': 'Los Angeles',
   'state': 'CA',
   'latitude': '34.087739',
   'longitude': '-118.202558',
   'FIPScounty': '6037',
   'useCode': 'SingleFamily',
   'taxAssessmentYear': '2019',
   'taxAssessment': '417145.0',
   'yearBuilt': '2002',
   'lotSizeSqFt': '5016',
   'finishedSqFt': '1844',
   'bathrooms': '3.0',
   'bedrooms': '3',
   'lastSoldDate': '11/06/2001',
   'lastSoldPrice': '315000',
   'amount': '822316',
   'last-updated': '10/26/2019',
   'valueChange': '25890',
   'low': '772977',
   'high': '871655',
   'percentile': '0',
   'zindexValue': '535,500'}},
 {'1020 MONTECITO DR LOS ANGELES CA 90031': {'zpid': '20689074',
   'street': '1020 Montecito Dr',
   'zipcode': '90031',
   'city': 'Los Angeles',
   'state': 'CA',
   'latitude': '34.087427',
   'longitude': '-118.203251',
   'FIPScounty': '6037',
   'useCode': 'SingleFamily',
   'taxAssessmen

In [93]:
Zillow_Database=[]
for i in Main_List:
    temp_dataframe=pd.DataFrame.from_dict(i.values())
    temp_dataframe["Full_Address"]=i.keys()
    temp_dataframe=temp_dataframe.set_index(temp_dataframe["Full_Address"])
    Zillow_Database.append(temp_dataframe)
result = pd.concat(Zillow_Database,sort=True)
result
   # for attributes in i.values():
    #    print(attributes.values())

,FIPScounty,Full_Address,amount,bathrooms,bedrooms,city,finishedSqFt,high,last-updated,lastSoldDate,lastSoldPrice,latitude,longitude,lotSizeSqFt,low,percentile,state,street,taxAssessment,taxAssessmentYear,totalRooms,useCode,valueChange,yearBuilt,zindexValue,zipcode,zpid
Full_Address,,,,,,,,,,,,,,,,,,,,,,,,,,,
1007 MONTECITO DR LOS ANGELES CA 90031,6037,1007 MONTECITO DR LOS ANGELES CA 90031,822316,3.0,3,Los Angeles,1844,871655,10/26/2019,11/06/2001,315000,34.087739,-118.202558,5016,772977,0,CA,1007 Montecito Dr,417145.0,2019,NaN,SingleFamily,25890,2002,"535,500",90031,20689097
1020 MONTECITO DR LOS ANGELES CA 90031,6037,1020 MONTECITO DR LOS ANGELES CA 90031,801818,2.0,3,Los Angeles,1822,857945,10/26/2019,NaN,NaN,34.087427,-118.203251,7648,737673,0,CA,1020 Montecito Dr,219621.0,2019,NaN,SingleFamily,21416,1991,"535,500",90031,20689074
1072 MONTECITO DR LOS ANGELES CA 90031,6037,1072 MONTECITO DR LOS ANGELES CA 90031,804888,3.0,3,Los Angeles,2068,869279,10/26/2019,11/12/2014,90000,34.086643,-118.201981,5200,740497,0,CA,1072 Montecito Dr,325598.0,2019,NaN,SingleFamily,16206,2017,"535,500",90031,20689138
1092 MONTECITO DR LOS ANGELES CA 90031,6037,1092 MONTECITO DR LOS ANGELES CA 90031,991635,3.0,4,Los Angeles,1581,1110631,10/26/2019,05/05/2017,885000,34.087098,-118.201609,5200,892472,0,CA,1092 Montecito Dr,920754.0,2019,NaN,Condominium,2286,1965,"535,500",90031,20689133
1196 MONTECITO DR LOS ANGELES CA 90031,6037,1196 MONTECITO DR LOS ANGELES CA 90031,993155,2.0,2,Los Angeles,1850,1042813,10/26/2019,NaN,NaN,34.087526,-118.199365,3070,943497,0,CA,1196 Montecito Dr,420326.0,2019,NaN,SingleFamily,68,2006,"535,500",90031,82872725
1226 MONTECITO DR LOS ANGELES CA 90031,6037,1226 MONTECITO DR LOS ANGELES CA 90031,764688,3.0,3,Los Angeles,1471,833510,10/26/2019,05/31/2013,490000,34.087346,-118.198436,5924,703513,0,CA,1226 Montecito Dr,540908.0,2019,NaN,SingleFamily,-7486,1993,"535,500",90031,20689168
122 AVENUE 23 LOS ANGELES CA 90031,NaN,122 AVENUE 23 LOS ANGELES CA 90031,738157,2.5,3,Los Angeles,1650,797210,10/26/2019,NaN,NaN,34.076503,-118.217979,20000,656960,0,CA,122 N Avenue 23,NaN,NaN,3,Townhouse,12429,2008,"489,200",90031,2112800844
122 AVENUE 23 LOS ANGELES CA 90031,NaN,122 AVENUE 23 LOS ANGELES CA 90031,738157,2.5,3,Los Angeles,1650,797210,10/26/2019,NaN,NaN,34.076503,-118.217979,20000,656960,0,CA,122 N Avenue 23,NaN,NaN,3,Townhouse,12429,2008,"489,200",90031,2112800844
122 AVENUE 23 LOS ANGELES CA 90031,NaN,122 AVENUE 23 LOS ANGELES CA 90031,738157,2.5,3,Los Angeles,1650,797210,10/26/2019,NaN,NaN,34.076503,-118.217979,20000,656960,0,CA,122 N Avenue 23,NaN,NaN,3,Townhouse,12429,2008,"489,200",90031,2112800844


In [76]:
#pd.DataFrame(data=[j.values() for i in Main_List for j in i.values()])
#pd.DataFrame(data=[j.values() for i in Main_List for j in i.values()],columns=[j.keys() for i in Main_List for j in i.values()])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
0,20689097,1007 Montecito Dr,90031,Los Angeles,CA,34.087739,-118.202558,6037,SingleFamily,2019,417145.0,2002,5016,1844,3.0,3,11/06/2001,315000,822316,10/26/2019,25890,772977,871655,0,"535,500",None
1,20689074,1020 Montecito Dr,90031,Los Angeles,CA,34.087427,-118.203251,6037,SingleFamily,2019,219621.0,1991,7648,1822,2.0,3,801818,10/26/2019,21416,737673,857945,0,"535,500",None,None,None
2,20689138,1072 Montecito Dr,90031,Los Angeles,CA,34.086643,-118.201981,6037,SingleFamily,2019,325598.0,2017,5200,2068,3.0,3,11/12/2014,90000,804888,10/26/2019,16206,740497,869279,0,"535,500",None
3,20689133,1092 Montecito Dr,90031,Los Angeles,CA,34.087098,-118.201609,6037,Condominium,2019,920754.0,1965,5200,1581,3.0,4,05/05/2017,885000,991635,10/26/2019,2286,892472,1110631,0,"535,500",None
4,82872725,1196 Montecito Dr,90031,Los Angeles,CA,34.087526,-118.199365,6037,SingleFamily,2019,420326.0,2006,3070,1850,2.0,2,993155,10/26/2019,68,943497,1042813,0,"535,500",None,None,None
5,20689168,1226 Montecito Dr,90031,Los Angeles,CA,34.087346,-118.198436,6037,SingleFamily,2019,540908.0,1993,5924,1471,3.0,3,05/31/2013,490000,764688,10/26/2019,-7486,703513,833510,0,"535,500",None
6,2112800844,122 N Avenue 23,90031,Los Angeles,CA,34.076503,-118.217979,Townhouse,2008,20000,1650,2.5,3,3,738157,10/26/2019,12429,656960,797210,0,"489,200",None,None,None,None,None
7,2112800844,122 N Avenue 23,90031,Los Angeles,CA,34.076503,-118.217979,Townhouse,2008,20000,1650,2.5,3,3,738157,10/26/2019,12429,656960,797210,0,"489,200",None,None,None,None,None
8,2112800844,122 N Avenue 23,90031,Los Angeles,CA,34.076503,-118.217979,Townhouse,2008,20000,1650,2.5,3,3,738157,10/26/2019,12429,656960,797210,0,"489,200",None,None,None,None,None
9,2112800844,122 N Avenue 23,90031,Los Angeles,CA,34.076503,-118.217979,Townhouse,2008,20000,1650,2.5,3,3,738157,10/26/2019,12429,656960,797210,0,"489,200",None,None,None,None,None
